# Calibration for $Q$

### Initialization

In [1]:
using DrWatson
@quickactivate "wrips-code"

In [2]:
using Ripserer, PersistenceDiagrams, PersistenceDiagramsBase
using InteractiveUtils, MLDataUtils, JLD2

In [3]:
using Random, Distributions, Parameters, Pipe, ProgressMeter, Plots
using Distances, LinearAlgebra, Statistics, LazySets, Roots

In [4]:
include(srcdir("wRips.jl"))
import Main.wRips

In [ ]:
plot_par = wRips.plot_params(alpha = 0.3)
theme(:dao)

### Data

In [ ]:
Random.seed!(2021)

n = 500
m = rand([50:1:150]..., 1)

l = 1.5
win = (-l, l, -l, l)
noise = wRips.randMClust(m..., window = win, λ1 = 2, λ2 = 10, r = 0.05)
# noise = wRips.randUnif(m..., a = -1, b = 1)
signal = 2.5 .* wRips.randCircle(n, σ = 0.05)


X = [signal; noise]
Xn_signal = wRips._ArrayOfTuples_to_ArrayOfVectors(signal)

Xn = wRips._ArrayOfTuples_to_ArrayOfVectors(X);

In [ ]:
Q_seq = range(10, 300, step = 10)
nQ = length(Q_seq)
num_iter = 20

In [ ]:
S = [[Dict() for _ = 1:num_iter] for _ in Q_seq];
@showprogress for i in 1:length(Q_seq)
    for j = 1:M
        dnq = wRips.momdist(Xn, Q_seq[i])
        w_momdist = wRips.fit(Xn, dnq)
        D_momdist = wRips.wrips(Xn, w = w_momdist, p = 1)
        S[i][j] = Dict("pers mom" => D_momdist[2] .|> persistence |> maximum)
    end
end

### Method $\#1$

In [ ]:
method1 = D -> D |> persistence |> maximum

In [ ]:
y = [  ]